In [1]:
from estnltk import EnvelopingBaseSpan
from estnltk import Annotation, Span, EnvelopingSpan, Layer, Text, Tagger

# Low-level layer operations
## Create a layer
```python
Layer(name: str,
      attributes: Sequence[str] = (),
      secondary_attributes: Sequence[str] = (),
      text_object: Text = None,
      parent: str = None,
      enveloping: str = None,
      ambiguous: bool = False,
      default_values: dict = None
      )
```
**name**: the name of the layer, required argument<br/>
**attributes**: the list of the attributes of the layer<br/>
**secondary_attributes**: the list of layer's attributes which will be skipped while comparing two layers<br/>
**text_object**: the `Text` object of the layer<br/>
**parent**: the name of the parent layer<br/>
**enveloping**: the layer is enveloping<br/>
**ambiguous**: the layer is ambiguous<br/>
**defauld_values**: default values of the attributes

If `parent is not None`, then `enveloping is None`.<br>
If `enveloping is not None`, then `parent is None`.


`secondary_attributes` are attributes that contain redundant information already present in `attributes`. For instance, attributes referring to parent and children spans in the syntax layer are marked as `secondary_attributes`, because they are recursive and comparing recursive spans leads to an (infinite) recursion error. However, such comparison is also not required, because other non-recursive attributes of the syntax layer already convey the information about parent-child relations between words.

### Has no parent, is not enveloping, is  not ambiguous

In [2]:
text = Text('Tere, maailm!')

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['attr_1', 'attr_2']
              )

layer.add_annotation(( 0,  4), attr_1='TERE', attr_2='lambda a: a.layer[1]')
layer.add_annotation(( 6, 12), attr_1='MAAILM', attr_2='lambda a: 12+88')
layer.add_annotation((12, 13), attr_1='!')

layer

Layer(name='test_layer', attributes=('attr_1', 'attr_2'), spans=SL[Span('Tere', [{'attr_1': 'TERE', 'attr_2': 'lambda a: a.layer[1]'}]),
Span('maailm', [{'attr_1': 'MAAILM', 'attr_2': 'lambda a: 12+88'}]),
Span('!', [{'attr_1': '!', 'attr_2': None}])])

### Has no parent, is not enveloping, is ambiguous
The code below differs from the code above by one line:
```python
ambiguous=True
```
Note that in this case equal spans are grouped together.

In [3]:
text = Text('Tere, maailm!')

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['attr_1'],
              ambiguous=True
              )
layer.add_annotation((0,  4), attr_1='TERE')
layer.add_annotation((6, 12), attr_1='MAAILM')
layer.add_annotation((6, 12), attr_1='MaaIlm')

layer

Layer(name='test_layer', attributes=('attr_1',), spans=SL[Span('Tere', [{'attr_1': 'TERE'}]),
Span('maailm', [{'attr_1': 'MAAILM'}, {'attr_1': 'MaaIlm'}])])

### Has a parent, is not enveloping, is  ambiguous

In [4]:
text = Text('Tere, maailm!').tag_layer(['words'])

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['a'],
              parent='words', 
              ambiguous=True
              )
for word in text.words:
    span = Span(base_span=word.base_span, layer=layer)

    span.add_annotation(a=1)
    span.add_annotation(a=2)
    
    layer.add_span(span)

layer

Layer(name='test_layer', attributes=('a',), spans=SL[Span('Tere', [{'a': 1}, {'a': 2}]),
Span(',', [{'a': 1}, {'a': 2}]),
Span('maailm', [{'a': 1}, {'a': 2}]),
Span('!', [{'a': 1}, {'a': 2}])])

### Does not have a parent, is enveloping, is not  ambiguous

In [5]:
t = 'Kolmkümmend kolm on suurem kui kaheksateist.'
text = Text(t)
text.tag_layer(['words'])

layer = Layer(name='test_layer',
              text_object=text,
              enveloping='words',
              attributes=['value'],
              ambiguous=False)

span = EnvelopingSpan(EnvelopingBaseSpan(s.base_span for s in text.words[0:2]), layer=layer)
span.add_annotation(value=33)
layer.add_span(span)

span = EnvelopingSpan(EnvelopingBaseSpan(([text.words[1].base_span, text.words[4].base_span])), layer=layer)
span.add_annotation(value=0)
layer.add_span(span)

span = EnvelopingSpan(EnvelopingBaseSpan((text.words[5].base_span, )), layer=layer)
span.add_annotation(value='18')
layer.add_span(span)

layer

Layer(name='test_layer', attributes=('value',), spans=SL[EnvelopingSpan(['Kolmkümmend', 'kolm'], [{'value': 33}]),
EnvelopingSpan(['kolm', 'kui'], [{'value': 0}]),
EnvelopingSpan(['kaheksateist'], [{'value': '18'}])])

The next demo phrase tagger marks all consequtive uppercase words inside a sentence.

In [6]:
import itertools
from operator import itemgetter
from itertools import groupby
        
class UppercasePhraseTagger(Tagger):
    """Tag uppercase phrases

    """
    conf_param = []
    
    def __init__(self):
        self.input_layers=['words', 'sentences']
        self.output_layer='uppercasephrase'
        self.output_attributes=['phrasetext', 'tag']

    def _make_layer_template(self) -> Layer:
        return Layer(name=self.output_layer, text_object=None, 
                     enveloping='words', attributes=self.output_attributes)
        
    def _make_layer(self, text: Text, layers, status) -> Layer:
        layer = self._make_layer_template()
        layer.text_object=text

        uppercases = []
        prevstart = 0
        for sentence in text.sentences:
            for idx, word in enumerate(sentence.words):
                if word.text.upper() == word.text and word.text.lower() != word.text:
                    uppercases.append((idx + prevstart, word))
            prevstart += len(sentence)

        rs = []
        for k, g in groupby(enumerate(uppercases), lambda i: i[0] - i[1][0]):
            r = map(itemgetter(1), g)
            rs.append(list(r))

        spans = [[j for _, j in i] for i in rs if len(i) > 1]

        for idx, s in enumerate(spans):
            layer.add_annotation(s, phrasetext = ' '.join([i.text for i in s]).lower(), tag=idx)

        return layer


tagger = UppercasePhraseTagger()
text = Text('Minu KARU ON PUNANE. MIS värvi SINU KARU on? Kuidas PALUN?')
text.tag_layer(['words', 'sentences'])
tagger.tag(text)

Text(text='Minu KARU ON PUNANE. MIS värvi SINU KARU on? Kuidas PALUN?')

In [7]:
text.uppercasephrase

Layer(name='uppercasephrase', attributes=('phrasetext', 'tag'), spans=SL[EnvelopingSpan(['KARU', 'ON', 'PUNANE'], [{'phrasetext': 'karu on punane', 'tag': 0}]),
EnvelopingSpan(['SINU', 'KARU'], [{'phrasetext': 'sinu karu', 'tag': 1}])])

In [8]:
text.tag_layer(['morph_analysis'])

print(text.phrasetext)
#['karu on punane', 'sinu karu']

print(text.uppercasephrase.lemma)
# [[['karu'], ['olema', 'olema'], ['punane']], [['sina'], ['karu']]]

print([i.text for i in text.words if i not in list(itertools.chain(*text.uppercasephrase.spans))])
# ['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']

print([i.text for i in text.words if i not in list(itertools.chain(*text.uppercasephrase.spans))])
# ['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']

mapping = {i: j for j in text.uppercasephrase for i in j.base_span}
print([mapping[i.base_span].tag if i.base_span in mapping else i.text for i in text.words])
# ['Minu', 0, 0, 0, '.', 'MIS', 'värvi', 1, 1, 'on', '?', 'Kuidas', 'PALUN', '?']

['karu on punane', 'sinu karu']
[AmbiguousAttributeList([['karu'], ['olema', 'olema'], ['punane']], ('lemma',)), AmbiguousAttributeList([['sina'], ['karu']], ('lemma',))]
['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']
['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']
['Minu', 0, 0, 0, '.', 'MIS', 'värvi', 1, 1, 'on', '?', 'Kuidas', 'PALUN', '?']


Here is an example how to create a new layer from an enveloping layer.

In [9]:
text = Text('Esimene lause? Teine lause! Kolmas lause.').tag_layer('sentences')
sentence_types = {'.': 'statement', '?': 'question', '!': 'exclamation'}

layer = Layer(name='sentence_type', enveloping=text.sentences.enveloping, attributes=('type',), text_object=text)
for s in text['sentences']:
    layer.add_annotation( s.base_span, type=sentence_types.get(s[-1].text) )

layer

Layer(name='sentence_type', attributes=('type',), spans=SL[EnvelopingSpan(['Esimene', 'lause', '?'], [{'type': 'question'}]),
EnvelopingSpan(['Teine', 'lause', '!'], [{'type': 'exclamation'}]),
EnvelopingSpan(['Kolmas', 'lause', '.'], [{'type': 'statement'}])])

## Descendant layers and ancestor layers

Descendant layers are downward in the chain of `parent` and `enveloping` relations, ancestor layers are upward in the tree of `parent` and `enveloping` relations.

In [10]:
text = Text('')

layer_1 = Layer('layer_1', text_object=text)
layer_2 = Layer('layer_2', text_object=text, parent='layer_1')
layer_3 = Layer('layer_3', text_object=text, parent='layer_2')
layer_4 = Layer('layer_4', text_object=text, enveloping='layer_2')
layer_5 = Layer('layer_5', text_object=text, enveloping='layer_2')
layer_6 = Layer('layer_6', text_object=text, parent='layer_5')

text.add_layer(layer_1)
text.add_layer(layer_2)
text.add_layer(layer_3)
text.add_layer(layer_4)
text.add_layer(layer_5)
text.add_layer(layer_6)

layer_2.descendant_layers()

['layer_3', 'layer_4', 'layer_5', 'layer_6']

In [11]:
layer_2.ancestor_layers()

['layer_1']